# Examples of CORDEX-CMIP6 cmorization

This notebook should show some examples of how a cmorized CORDEX-CMIP6 dataset could look like.

In [ ]:
!ls ../Tables

## Controlled Vocabulary

The current `CORDEX-CMIP6_CV.json` file contains the controlled vocabulary used by the CMOR3 library for rewriting the model output. Let's have a look:

In [ ]:
from IPython.display import JSON
import json
import pprint

# Opening JSON file
with open("../Tables/CORDEX-CMIP6_CV.json") as json_file:
    CV = json.load(json_file)["CV"]
    
JSON(CV['required_global_attributes'])
JSON(CV)